In [19]:
import urllib.request
import re
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import json
import requests
import json


handgun = "https://www.luckygunner.com/handgun"

In [20]:
# returns 2 list of http links separated by availability
def luckyGunner_availability(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    page_body = soup.body
    products = soup.find_all('div', class_='main')

    
    # list of all handgun ammo seperated by availability
    products = soup.find_all('div', class_='block-content block-no-padding')
    bulletsList = str(products).split('<div class="main-category-cols">')
    inStock = bulletsList[1]
    outofStock = bulletsList[2]

    inStockScrapedLinks = []
    outofStockScrapedLinks = []
    
    inStock = inStock.split("</li>")
    for inStockItems in inStock:
        inStockScrapedLinks.append(re.findall("(?P<url>https?://[^\s]+)", str(inStockItems)))


    outofStock = outofStock.split("</li>")
    for outofStockItems in outofStock:
        outofStockScrapedLinks.append(re.findall("(?P<url>https?://[^\s]+)", str(outofStockItems)))

    inStockCleanLinks = []
    outofStockCleanLinks = []
    

    for i in inStockScrapedLinks:
        cleanLink = ""
        for j in i:
            for k in j:
                if k != '"':
                    cleanLink += k
                    
                else:
                    inStockCleanLinks.append(cleanLink)
                    break
                
    for i in outofStockScrapedLinks:
        cleanLink = ""
        for j in i:
            for k in j:
                if k != '"':
                    cleanLink += k
                else:
                    outofStockCleanLinks.append(cleanLink)
                    break
    return inStockCleanLinks, outofStockCleanLinks

In [21]:
x = luckyGunner_availability(handgun)

In [22]:
def scrapeHandGunAmmo(link):
    link+="?limit=all"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    html_response = requests.get(url=link, headers = headers)
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "lxml")

    names = []
    prices = []
    aLinks = []

    # list of html elements
    names_list = soup.find_all("h3", class_="product-name")
    prices_list = soup.find_all("div", class_="price-box")
    caliber = soup.find_all("div", class_="category-title")
    caliber = caliber[0].text
    soup_links = soup.find_all("li", class_="item")

    


    
    for i in prices_list:
        if "Special Price" in str(i):
            sp = i.find("p", class_="special-price")
            sp = sp.find("span", class_="price")
           
            num = ""
            for j in sp.text:
                if str(j) != " " and str(j) != "$":
                    num += str(j)
            prices.append(num)
            num = ""      

        else:
            num = ""
            for j in i.find("span").text:
                if str(j) != " " and str(j) != "$":
                    num += str(j)
            prices.append(num)
            num = ""
                 
                 
    for i in names_list:
        names.append(i.text)



    
    for i in soup_links:
        x = (i.find('a'))
        aLinks.append(x["href"])
  
   
        
        

    try:
        ammoData = pd.DataFrame(
            {"name": names, "price": prices, "caliber": caliber, "links": aLinks}
        )
        return ammoData
        
    except:
        print("Check link: " + link)
        # soup_links = soup.find_all("li", class_="item")
        # for i in soup_links:
        #     x = (i.find('a'))
        #     print(x["href"])
        # print("----------------")
        # # print(len(names))
        # # print("----------------")
        # # print(len(prices))
        # # print("----------------")
        # # print(caliber)
        # # print("----------------")
        # # print(aLinks)

In [23]:
list_of_names = []
name = []
list_of_prices = []
price = []
list_of_links = []
links = []
list_of_calibers = []
caliber = []

for i in x[0]:

    names = []
    prices = []
    caliber = []
    links = []
    y = scrapeHandGunAmmo(i)
    # print(y)
    # print("------------------------------------------\n")

    for j in y["name"]:
        names.append(j.replace("\n\n", ""))
    list_of_names.append(names)
    
    for j in y["price"]:
        prices.append(j.replace("\n", ""))
    list_of_prices.append(prices)

    for j in y["caliber"]:
        caliber.append(j)
    list_of_calibers.append(caliber)

    for j in y["links"]:
        links.append(j)
    list_of_links.append(links)



        

    



In [24]:
for i in list_of_names:
    for j in i:
        name.append(j)
for i in list_of_prices:
    for j in i:
        price.append(j)
for i in list_of_calibers:
    for j in i:
        caliber.append(j)
for i in list_of_links:
    for j in i:
        links.append(j)


KeyboardInterrupt: 